<a href="https://colab.research.google.com/github/ArPaWi/ML_PadiCare/blob/arya/dev_padicare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **NOTEBOOK PADI CARE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 966.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os
import random
import numpy as np
from io import BytesIO

# Plotting and dealing with images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf

# Interactive widgets
from ipywidgets import widgets

# MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
BASE_DIR = r'drive/MyDrive/data'

train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')

# Directory with training pictures
train_brownspot_dir = os.path.join(train_dir, 'brownspot')
train_healthy_dir = os.path.join(train_dir, 'healthy')
train_hispa_dir = os.path.join(train_dir, 'hispa')
train_leafblast_dir = os.path.join(train_dir, 'leafblast')

# Directory with validation pictures
validation_brownspot_dir = os.path.join(validation_dir, 'brownspot')
validation_healthy_dir = os.path.join(validation_dir, 'healthy')
validation_hispa_dir = os.path.join(validation_dir, 'hispa')
validation_leafblast_dir = os.path.join(validation_dir, 'leafblast')

# Function to print directory contents
def print_directory_contents(directory, description):
    if os.path.exists(directory):
        print(f"\nContents of {description}: {os.listdir(directory)}")
    else:
        print(f"\nDirectory {description} does not exist: {directory}")

# Check base directory
print_directory_contents(BASE_DIR, "base directory")

# Check train and validation directories
print_directory_contents(train_dir, "train directory")
print_directory_contents(validation_dir, "validation directory")

# Check each class directory
print_directory_contents(train_brownspot_dir, "train/brownspot directory")
print_directory_contents(train_healthy_dir, "train/healthy directory")
print_directory_contents(train_hispa_dir, "train/hispa directory")
print_directory_contents(train_leafblast_dir, "train/leafblast directory")

print_directory_contents(validation_brownspot_dir, "validation/brownspot directory")
print_directory_contents(validation_healthy_dir, "validation/healthy directory")
print_directory_contents(validation_hispa_dir, "validation/hispa directory")
print_directory_contents(validation_leafblast_dir, "validation/leafblast directory")


Contents of base directory: ['train', 'validation']

Contents of train directory: ['leafblast', 'healthy', 'brownspot', 'hispa']

Contents of validation directory: ['hispa', 'brownspot', 'leafblast', 'healthy']

Contents of train/brownspot directory: ['train_brownspot323.jpg', 'train_brownspot269.jpg', 'train_brownspot127.jpg', 'train_brownspot12.jpg', 'train_brownspot252.jpg', 'train_brownspot207.jpg', 'train_brownspot18.jpg', 'train_brownspot316.jpg', 'train_brownspot184.jpg', 'train_brownspot188.jpg', 'train_brownspot140.jpg', 'train_brownspot221.jpg', 'train_brownspot236.jpg', 'train_brownspot113.jpg', 'train_brownspot275.jpg', 'train_brownspot126.jpg', 'train_brownspot267.jpg', 'train_brownspot158.jpg', 'train_brownspot24.jpg', 'train_brownspot309.jpg', 'train_brownspot186.jpg', 'train_brownspot228.jpg', 'train_brownspot19.jpg', 'train_brownspot105.jpg', 'train_brownspot151.jpg', 'train_brownspot208.jpg', 'train_brownspot117.jpg', 'train_brownspot16.jpg', 'train_brownspot194.jpg'

In [ ]:
print(f'total training brownspot images: {len(os.listdir(train_brownspot_dir))}')
print(f'total training healthy images: {len(os.listdir(train_healthy_dir))}')
print(f'total training hispa images: {len(os.listdir(train_hispa_dir))}')
print(f'total training leafblast images: {len(os.listdir(train_leafblast_dir))}')

print(f'total validation brownspot images: {len(os.listdir(validation_brownspot_dir))}')
print(f'total validation healthy images: {len(os.listdir(validation_healthy_dir))}')
print(f'total validation hispa images: {len(os.listdir(validation_hispa_dir))}')
print(f'total validation leafblast images: {len(os.listdir(validation_leafblast_dir))}')

total training brownspot images: 450
total training healthy images: 450
total training hispa images: 450
total training leafblast images: 450
total validation brownspot images: 113
total validation healthy images: 113
total validation hispa images: 113
total validation leafblast images: 113


In [ ]:
base_model = MobileNetV2(
    input_shape=(256, 256, 3),
    include_top=False,  # Tidak menyertakan lapisan klasifikasi bawaan
    weights='imagenet'  # Menggunakan bobot pretrained
)
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

<ipython-input-32-7f08db5e34e6>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(4, activation='softmax')
])


In [ ]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 8, 8, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,178,564 (12.13 MB)

 Trainable params: 2,782,020 (10.61 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Membuat generator dengan augmentasi
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisasi pixel (0-1)
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Dataset training dengan augmentasi
train_dataset_augmented = datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),  # Ukuran gambar
    batch_size=30,          # Ukuran batch
    class_mode='categorical'
)

# Dataset validasi (tanpa augmentasi)
val_datagen = ImageDataGenerator(rescale=1./255)
validation_dataset = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),
    batch_size=30,
    class_mode='categorical'
)

Found 1800 images belonging to 4 classes.
Found 452 images belonging to 4 classes.


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [ ]:
history = model.fit(
    train_dataset_augmented,
    epochs=15,
    validation_data=validation_dataset,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step - accuracy: 0.3664 - loss: 6.3152 - val_accuracy: 0.3274 - val_loss: 6.1101 - learning_rate: 1.0000e-04
Epoch 2/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.5595 - loss: 5.4845 - val_accuracy: 0.3319 - val_loss: 5.9516 - learning_rate: 1.0000e-04
Epoch 3/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 256s 4s/step - accuracy: 0.6314 - loss: 4.9000 - val_accuracy: 0.3407 - val_loss: 5.8270 - learning_rate: 1.0000e-04
Epoch 4/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 239s 4s/step - accuracy: 0.6998 - loss: 4.3926 - val_accuracy: 0.3186 - val_loss: 5.8242 - learning_rate: 1.0000e-04
Epoch 5/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - accuracy: 0.7004 - loss: 4.0078 - val_accuracy: 0.3075 - val_loss: 6.0707 - learning_rate: 1.0000e-04
Epoch 6/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.7260 - loss: 3.6346 - val_accuracy: 0.3053 - val_loss: 5.5704 - learning_rate: 1.0000e-04
Epoch 7/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 254s 4s/step - accuracy: 0.7173 - loss: 3.3